In [1]:
import json,sys,requests
import pandas as pd
from datetime import datetime
import random

In [2]:
def getPluginTemplate():
    keys = ['timestamp','name','value','metadata']
    metadataKeys = ['node','host','plugin','camera']
    dataAPISchema = dict.fromkeys(keys)
    dataAPISchema['metadata'] = dict.fromkeys(metadataKeys)
    return dataAPISchema

template = getPluginTemplate()
print(json.dumps(template, indent=4))


{
    "timestamp": null,
    "name": null,
    "value": null,
    "metadata": {
        "node": null,
        "host": null,
        "plugin": null,
        "camera": null
    }
}


In [3]:
strDate = '4/1/2021'
endDate = '4/14/2021'
freq = 'H'# T:minute,H:hourly,D:daily,
dateIndex = pd.date_range(start=strDate, end=endDate, freq=freq,tz='US/Pacific')
dateIndex

DatetimeIndex(['2021-04-01 00:00:00-07:00', '2021-04-01 01:00:00-07:00',
               '2021-04-01 02:00:00-07:00', '2021-04-01 03:00:00-07:00',
               '2021-04-01 04:00:00-07:00', '2021-04-01 05:00:00-07:00',
               '2021-04-01 06:00:00-07:00', '2021-04-01 07:00:00-07:00',
               '2021-04-01 08:00:00-07:00', '2021-04-01 09:00:00-07:00',
               ...
               '2021-04-13 15:00:00-07:00', '2021-04-13 16:00:00-07:00',
               '2021-04-13 17:00:00-07:00', '2021-04-13 18:00:00-07:00',
               '2021-04-13 19:00:00-07:00', '2021-04-13 20:00:00-07:00',
               '2021-04-13 21:00:00-07:00', '2021-04-13 22:00:00-07:00',
               '2021-04-13 23:00:00-07:00', '2021-04-14 00:00:00-07:00'],
              dtype='datetime64[ns, US/Pacific]', length=313, freq='H')

In [4]:
# get node data from endpoint
nodeURL = 'https://sage-node-api.nautilus.optiputer.net/api/v1/nodes-data'
response = requests.request("GET", nodeURL)
nodeJSON = response.json()
nodeDataDF = pd.DataFrame.from_dict(nodeJSON['data'], orient='columns')

uniqueNodeIDs = nodeDataDF.id.unique()

subsetOfUniqueNodeIDs = set()
totalNodeIDs = 3

while len(subsetOfUniqueNodeIDs) < totalNodeIDs:
    nodeIDPickedRand = random.choice(uniqueNodeIDs)
    subsetOfUniqueNodeIDs.add(nodeIDPickedRand)

subsetOfUniqueNodeIDs = list(subsetOfUniqueNodeIDs)
print(uniqueNodeIDs)
print(subsetOfUniqueNodeIDs)

['4cd98fc4d2a8' '4cd98fc67b75' '4cd98fc68d9c' '4cd98fc67734'
 '4cd98fc67a3d' '4cd98fc67e76' '4cd98fc68549' '4cd98fc686c9'
 '4cd98fb00025' '4cd98fa8be25' '4cd98fadebf3' '4cd98fb554db'
 '2cea7f5a2f72' '2cea7f5a0a3d']
['2cea7f5a0a3d', '4cd98fa8be25', '4cd98fc686c9']


In [5]:
randomGenList = []
name = 'env.temperature.gen'
plugin = 'metsense:1.0.2'

for date in dateIndex:
    template = getPluginTemplate()
    template['timestamp']
    template['name'] = name
    template['value'] = random.random()
    template['timestamp'] = pd.Timestamp(date).tz_convert('utc').strftime('%Y-%m-%dT%H:%M:%SZ')
    template['metadata']['node'] = random.choice(subsetOfUniqueNodeIDs)
    template['metadata']['plugin'] = plugin
    randomGenList.append(template)

In [6]:
data = pd.DataFrame(randomGenList)
data

,timestamp,name,value,metadata
0,2021-04-01T07:00:00Z,env.temperature.gen,0.350728,"{'node': '4cd98fa8be25', 'host': None, 'plugin..."
1,2021-04-01T08:00:00Z,env.temperature.gen,0.967763,"{'node': '4cd98fc686c9', 'host': None, 'plugin..."
2,2021-04-01T09:00:00Z,env.temperature.gen,0.135185,"{'node': '4cd98fa8be25', 'host': None, 'plugin..."
3,2021-04-01T10:00:00Z,env.temperature.gen,0.691703,"{'node': '4cd98fa8be25', 'host': None, 'plugin..."
4,2021-04-01T11:00:00Z,env.temperature.gen,0.077814,"{'node': '2cea7f5a0a3d', 'host': None, 'plugin..."
...,...,...,...,...
308,2021-04-14T03:00:00Z,env.temperature.gen,0.177409,"{'node': '4cd98fa8be25', 'host': None, 'plugin..."
309,2021-04-14T04:00:00Z,env.temperature.gen,0.088100,"{'node': '4cd98fc686c9', 'host': None, 'plugin..."
310,2021-04-14T05:00:00Z,env.temperature.gen,0.517077,"{'node': '2cea7f5a0a3d', 'host': None, 'plugin..."
311,2021-04-14T06:00:00Z,env.temperature.gen,0.395054,"{'node': '2cea7f5a0a3d', 'host': None, 'plugin..."


In [7]:
jsonData = data.to_json(orient='records')
jsonData

'[{"timestamp":"2021-04-01T07:00:00Z","name":"env.temperature.gen","value":0.3507279307,"metadata":{"node":"4cd98fa8be25","host":null,"plugin":"metsense:1.0.2","camera":null}},{"timestamp":"2021-04-01T08:00:00Z","name":"env.temperature.gen","value":0.9677626058,"metadata":{"node":"4cd98fc686c9","host":null,"plugin":"metsense:1.0.2","camera":null}},{"timestamp":"2021-04-01T09:00:00Z","name":"env.temperature.gen","value":0.1351853549,"metadata":{"node":"4cd98fa8be25","host":null,"plugin":"metsense:1.0.2","camera":null}},{"timestamp":"2021-04-01T10:00:00Z","name":"env.temperature.gen","value":0.6917033296,"metadata":{"node":"4cd98fa8be25","host":null,"plugin":"metsense:1.0.2","camera":null}},{"timestamp":"2021-04-01T11:00:00Z","name":"env.temperature.gen","value":0.0778137201,"metadata":{"node":"2cea7f5a0a3d","host":null,"plugin":"metsense:1.0.2","camera":null}},{"timestamp":"2021-04-01T12:00:00Z","name":"env.temperature.gen","value":0.1863501961,"metadata":{"node":"2cea7f5a0a3d","host":n

In [8]:
data.to_json(orient='records',path_or_buf=r'pluginData.json')